In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [3]:
data = pd.read_csv('dataset/mail_data.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Spam'] = le.fit_transform(data['Category'])
data = data.drop('Category',axis=1)
data.head()

,Message,Spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
# !pip install nltk
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# data['Message'] = data['Message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# data['Message'][0]

'Go jurong point, crazy.. Available bugis n great world la e buffet... Cine got amore wat...'

In [6]:
X = data['Message']
y = data['Spam']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word

In [10]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
for el in X_train_seq[0]:
    print(word_dict[el], end=' ')

i thk u dun haf 2 hint in e forum already lor cos i told ron n darren is going 2 tell shuhui 

In [11]:
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')
X_train_pad[:5]
X_train_pad.shape

(4179, 20)

In [12]:
laenge_pads = 20
anz_woerter = 7982

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=20, input_length=laenge_pads))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 20)            159660    
                                                                 
 lstm (LSTM)                 (None, 400)               673600    
                                                                 
 dense (Dense)               (None, 1)                 401       
                                                                 
Total params: 833,661
Trainable params: 833,661
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))


Epoch 1/10
66/66 [==============================] - 10s 126ms/step - loss: 0.2229 - accuracy: 0.9234 - val_loss: 0.0998 - val_accuracy: 0.9727
Epoch 2/10
66/66 [==============================] - 7s 108ms/step - loss: 0.0469 - accuracy: 0.9873 - val_loss: 0.0654 - val_accuracy: 0.9777
Epoch 3/10
66/66 [==============================] - 8s 115ms/step - loss: 0.0212 - accuracy: 0.9959 - val_loss: 0.0731 - val_accuracy: 0.9799
Epoch 4/10
66/66 [==============================] - 8s 126ms/step - loss: 0.0112 - accuracy: 0.9976 - val_loss: 0.0890 - val_accuracy: 0.9756
Epoch 5/10
66/66 [==============================] - 9s 134ms/step - loss: 0.0051 - accuracy: 0.9993 - val_loss: 0.1331 - val_accuracy: 0.9763
Epoch 6/10
66/66 [==============================] - 9s 134ms/step - loss: 0.0028 - accuracy: 0.9995 - val_loss: 0.0954 - val_accuracy: 0.9777
Epoch 7/10
66/66 [==============================] - 9s 135ms/step - loss: 4.3507e-04 - accuracy: 1.0000 - val_loss: 0.1488 - val_accuracy: 0.9770
E

In [15]:
test = ["XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n="]
sms_seq = tokenizer.texts_to_sequences(test)

sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')
tokenizer.index_word
sms_pad
lstm_model.predict(sms_pad)

1/1 [==============================] - 0s 476ms/step


array([[0.99999684]], dtype=float32)

In [16]:
test = ["How are you"]
sms = tokenizer.texts_to_sequences(test)
sms_pad = pad_sequences(sms,maxlen=20,padding='post')
tokenizer.index_word
pred = lstm_model.predict(sms_pad)

1/1 [==============================] - 0s 23ms/step


array([[3.5879543e-06]], dtype=float32)

In [18]:
lstm_model.save('model.h5')

In [27]:
import pickle
tokenizer = pickle.dump(tokenizer,open('tokenizer.pkl','wb'))